# 🚀 AI-Powered Sales Intelligence Demo powered by Snowflake Cortex

## 🎯 Purpose
This notebook demonstrates how Snowball Analytics leverages Snowflake's Cortex AI capabilities to transform traditional CRM data into actionable sales intelligence. Experience conversational AI that answers complex sales questions, identifies priority accounts, and generates personalized outreach.

## 💡 Why do this?
- 🎯 **Account Prioritization**: AI-driven scoring to identify high-value opportunities and at-risk accounts
- 🤝 **Meeting Preparation**: Instant context and relationship history for prospect meetings
- 🏆 **Competitive Intelligence**: Extract competitor mentions and sentiment from sales activities
- 📈 **MEDDIC Analysis**: Automated identification of budget, timeline, and decision-maker signals
- ✉️ **Personalized Outreach**: Generate contextual follow-ups based on recent interactions
- 📊 **Pipeline Health**: Predict deal outcomes and recommended next actions

## 🛠️ Solution Components
Note: This demo uses SQL & Python to showcase Snowflake Cortex capabilities for sales intelligence

1. 📊 **Sales Data Exploration**
   - Interactive dashboards showing account activity and opportunity trends
   - Dynamic filtering by territory, industry, and deal stage

2. 🤖 **AI-Powered Sales Analysis**
   - Account summarization using `AI_AGG` across task history
   - Automated competitor detection with `AI_FILTER`
   - Multi-dimensional sentiment analysis with `ENTITY_SENTIMENT`
   - MEDDIC scoring and opportunity assessment

3. ⚙️ **Automated Sales Intelligence Pipeline**
   - Real-time processing of new sales activities
   - Automatic account health scoring updates
   - Competitor mention tracking and alerts

4. 🎯 **Conversational Sales Assistant**
   - Natural language queries for meeting preparation
   - Semantic search across sales history and customer interactions
   - Contextual recommendations and next best actions

## 🌟 Demo Scenarios
*"I am about to meet Barbara Davis at Hamburg Digital. Summarize our relationship history and suggest talking points."*

*"Which accounts mentioned competitors in Q3 and what was our response strategy?"*

*"Show me all deals where budget has been approved but we haven't closed yet."*

*"Generate a follow-up email for Hamburg Digital based on our latest technical deep dive."*


In [ ]:
# Core packages for sales analytics and visualization
import streamlit as st
import pandas as pd
import altair as alt
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go

# Snowflake session for AI-powered analytics
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
# Sales data configuration - update these if using your own CRM data
tasks_table = 'AI_SOL.SALES.TASKS'
accounts_table = 'AI_SOL.SALES.ACCOUNTS' 
opportunities_table = 'AI_SOL.SALES.OPPORTUNITIES'
contacts_table = 'AI_SOL.SALES.CONTACTS'

# Display table locations for reference
st.markdown(f"""
**Sales Data Sources:**
- 📋 Tasks: `{tasks_table}`
- 🏢 Accounts: `{accounts_table}`  
- 💰 Opportunities: `{opportunities_table}`
- 👥 Contacts: `{contacts_table}`
""")

In [ ]:
-- Get a high-level view of our sales data
-- This helps us understand the scope and timeframe of our demo data
SELECT 
    'Tasks' as data_type,
    COUNT(*) as record_count,
    MIN(activity_date) as earliest_date,
    MAX(activity_date) as latest_date
FROM {{tasks_table}}

UNION ALL

SELECT 
    'Opportunities' as data_type,
    COUNT(*) as record_count,
    MIN(close_date) as earliest_date,
    MAX(close_date) as latest_date
FROM {{opportunities_table}}

UNION ALL

SELECT 
    'Accounts' as data_type,
    COUNT(*) as record_count,
    MIN(created_date) as earliest_date,
    MAX(last_modified_date) as latest_date
FROM {{accounts_table}};

In [ ]:
## Interactive Sales Dashboard
## This provides executives and sales reps with key metrics and trends

st.header("📊 Sales Performance Dashboard")

# Get opportunity data for visualization
opps_df = session.sql(f"""
    SELECT 
        o.STAGE_NAME,
        o.CLOSE_DATE,
        o.AMOUNT,
        o.PROBABILITY,
        a.BILLING_COUNTRY as GEO,
        a.ANNUAL_REVENUE,
        a.NAME as ACCOUNT_NAME,
        o.LEAD_SOURCE
    FROM {opportunities_table} o
    JOIN {accounts_table} a ON o.ACCOUNT_ID = a.ID
    WHERE o.CLOSE_DATE >= '2024-01-01'
""").to_pandas()

# Convert CLOSE_DATE to datetime
opps_df['CLOSE_DATE'] = pd.to_datetime(opps_df['CLOSE_DATE'])

# Create form for dashboard filters
with st.form("dashboard_filters"):
    col1, col2, col3 = st.columns(3)
    
    with col1:
        selected_industries = st.multiselect(
            'Filter by Geo', 
            options=opps_df['GEO'].unique(),
            default=opps_df['GEO'].unique()
        )
    
    with col2:
        date_range = st.date_input(
            'Date Range',
            value=(opps_df['CLOSE_DATE'].min(), opps_df['CLOSE_DATE'].max()),
            min_value=opps_df['CLOSE_DATE'].min(),
            max_value=opps_df['CLOSE_DATE'].max()
        )
    
    with col3:
        min_deal_size = st.number_input(
            'Minimum Deal Size (€)', 
            min_value=0, 
            value=0,
            step=10000
        )
    
    submitted = st.form_submit_button("Update Dashboard")

if submitted or True:  # Show default on first load
    # Filter data based on selections
    filtered_df = opps_df[
        (opps_df['GEO'].isin(selected_industries)) &
        (opps_df['CLOSE_DATE'] >= pd.to_datetime(date_range[0])) &
        (opps_df['CLOSE_DATE'] <= pd.to_datetime(date_range[1])) &
        (opps_df['AMOUNT'] >= min_deal_size)
    ]
    
    # Key metrics row
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        total_pipeline = filtered_df['AMOUNT'].sum()
        st.metric("Total Pipeline", f"€{total_pipeline:,.0f}")
    
    with col2:
        closed_won = filtered_df[filtered_df['STAGE_NAME'] == 'Closed Won']['AMOUNT'].sum()
        st.metric("Closed Won", f"€{closed_won:,.0f}")
    
    with col3:
        avg_deal_size = filtered_df['AMOUNT'].mean()
        st.metric("Avg Deal Size", f"€{avg_deal_size:,.0f}")
    
    with col4:
        win_rate = len(filtered_df[filtered_df['STAGE_NAME'] == 'Closed Won']) / len(filtered_df) * 100
        st.metric("Win Rate", f"{win_rate:.1f}%")
    
    # Pipeline by stage chart
    stage_summary = filtered_df.groupby('STAGE_NAME')['AMOUNT'].sum().reset_index()
    
    fig_pipeline = px.bar(
        stage_summary, 
        x='STAGE_NAME', 
        y='AMOUNT',
        title='Pipeline by Stage',
        labels={'AMOUNT': 'Amount (€)', 'STAGE_NAME': 'Sales Stage'}
    )
    st.plotly_chart(fig_pipeline, use_container_width=True)
    
    # Monthly trend
    monthly_trend = filtered_df.groupby(filtered_df['CLOSE_DATE'].dt.to_period('M'))['AMOUNT'].sum().reset_index()
    monthly_trend['CLOSE_DATE'] = monthly_trend['CLOSE_DATE'].astype(str)
    
    fig_trend = px.line(
        monthly_trend,
        x='CLOSE_DATE',
        y='AMOUNT', 
        title='Monthly Pipeline Trend',
        labels={'AMOUNT': 'Amount (€)', 'CLOSE_DATE': 'Month'}
    )
    st.plotly_chart(fig_trend, use_container_width=True)


# 🎯 AI-Powered Account Intelligence

Now let's dive into the real power of Cortex AI for sales intelligence. We'll use `AI_AGG` to automatically summarize account relationships and identify key insights from our sales activities.

This is where Snowball Analytics transforms from traditional CRM reporting to intelligent sales assistance.


In [ ]:
-- AI-powered account relationship summary
-- This demonstrates how we can instantly prepare for customer meetings
-- by analysing all historical interactions and extracting key insights

SELECT 
    a.name as account_name,
    a.industry,
    a.annual_revenue,
    COUNT(t.id) as total_activities,
    AI_AGG(
        t.description,
        'Summarize the relationship history with this account. Highlight key pain points discussed, decision makers engaged, competitive mentions, and current status. Include any budget or timing indicators mentioned.'
    ) as relationship_summary
FROM {{accounts_table}} a
JOIN {{tasks_table}} t ON a.id = t.account_id
WHERE a.name = 'Hamburg Digital'
GROUP BY a.name, a.industry, a.annual_revenue;


In [ ]:
# Meeting Preparation Assistant
# This simulates the use case: "I'm about to meet Barbara Davis at Hamburg Digital"
# Note how "AISQL" is a callable python function too :)

from snowflake.snowpark.functions import col, count_distinct, max, ai_agg, concat, lit

st.header("🤝 Meeting Preparation Assistant")

# Create DataFrames from tables
accounts_df = session.table(accounts_table)
tasks_df = session.table(tasks_table)
opps_df = session.table(opportunities_table)
contacts_df = session.table(contacts_table)

# Create interactive meeting prep tool
with st.form("meeting_prep"):
    st.markdown("### Prepare for your next meeting with AI insights")
    
    # Get list of accounts for selection 
    accounts_list = (accounts_df.select('NAME').distinct().sort('NAME').to_pandas()['NAME'].tolist()
)
    
    selected_account = st.selectbox(
        "Select Account for Meeting Prep:",
        options=accounts_list,
        index=accounts_list.index('Hamburg Digital') if 'Hamburg Digital' in accounts_list else 0
    )
    
    prep_submitted = st.form_submit_button("Generate Meeting Brief")

if prep_submitted:
    # Get comprehensive account intelligence
    meeting_brief = (accounts_df
        .filter(col('NAME') == selected_account)
        .join(
            tasks_df.select(
                col('ID').alias('TASK_ID'),
                col('ACCOUNT_ID').alias('TASK_ACCOUNT_ID'),
                col('DESCRIPTION').alias('TASK_DESC'),
                col('ACTIVITY_DATE')
            ),
            accounts_df['ID'] == col('TASK_ACCOUNT_ID'),
            'left'
        )
        .join(
            opps_df.select(
                col('ID').alias('OPP_ID'),
                col('ACCOUNT_ID').alias('OPP_ACCOUNT_ID')
            ),
            accounts_df['ID'] == col('OPP_ACCOUNT_ID'),
            'left'
        )
        .group_by(accounts_df['NAME'], accounts_df['INDUSTRY'], accounts_df['ANNUAL_REVENUE'])
        .agg(
            count_distinct('TASK_ID').alias('TOTAL_ACTIVITIES'),
            count_distinct('OPP_ID').alias('TOTAL_OPPORTUNITIES'),
            max('ACTIVITY_DATE').alias('LAST_ACTIVITY_DATE'),
            ai_agg(
                col('TASK_DESC'),
                'Create a comprehensive meeting preparation brief. Include: 1) Recent activity summary 2) Key stakeholders and their roles 3) Pain points and business priorities 4) Competitive landscape mentions 5) Budget and timeline indicators 6) Recommended talking points and questions to ask. Format as executive brief.'
            ).alias('MEETING_BRIEF')
        )
        .collect()
    )
    
    if meeting_brief:
        brief = meeting_brief[0]
        
        st.success(f"Meeting Brief Generated for {selected_account}")
        
        # Display key metrics
        col1, col2, col3, col4 = st.columns(4)
        with col1:
            st.metric("Industry", brief['INDUSTRY'])
        with col2:
            st.metric("Annual Revenue", f"€{brief['ANNUAL_REVENUE']:,}")
        with col3:
            st.metric("Total Activities", brief['TOTAL_ACTIVITIES'])
        with col4:
            st.metric("Opportunities", brief['TOTAL_OPPORTUNITIES'])
        
        # Display AI-generated brief
        st.markdown("### 📋 AI-Generated Meeting Brief")
        st.markdown(brief['MEETING_BRIEF'])
        
        # Get recent contacts for this account
        recent_contacts = (contacts_df
            .join(accounts_df, contacts_df['ACCOUNT_ID'] == accounts_df['ID'])
            .filter(accounts_df['NAME'] == selected_account)
            .select(
                concat(contacts_df['FIRST_NAME'], lit(' '), contacts_df['LAST_NAME']).alias('CONTACT_NAME'),
                contacts_df['TITLE'],
                contacts_df['DEPARTMENT'],
                contacts_df['EMAIL']
            )
            .distinct()
            .to_pandas()
        )
        
        if not recent_contacts.empty:
            st.markdown("### 👥 Key Contacts")
            st.dataframe(recent_contacts, use_container_width=True)


# 🏆 Competitive Intelligence with AI_FILTER

One of the key challenges in B2B sales is tracking competitive mentions scattered across sales activities. `AI_FILTER` allows us to intelligently identify competitive threats and opportunities, even when competitors aren't explicitly named.

This goes beyond simple keyword matching - it understands context and intent.

In [ ]:
-- Intelligent competitor detection across all sales activities
-- This finds both explicit competitor mentions and implicit competitive scenarios

SELECT 
    t.id,
    a.name as account_name,
    a.industry,
    t.subject,
    t.activity_date,
    AI_FILTER(PROMPT('Does this sales activity mention competitors, competitive evaluation, or vendor comparison? Include mentions of RFPs, vendor selection, or "other solutions" being considered. {0}', t.description)) as has_competitive_mention,
    CASE 
        WHEN AI_FILTER(PROMPT('Does this mention a specific competitor by name? {0}', t.description)) THEN 'Named Competitor'
        WHEN AI_FILTER(PROMPT('Does this mention evaluating multiple vendors or solutions? {0}', t.description)) THEN 'Competitive Evaluation'
        WHEN AI_FILTER(PROMPT('Does this mention RFP or procurement process? {0}', t.description)) THEN 'Formal RFP Process'
        ELSE 'Other Competitive Signal'
    END as competitive_type,
    t.description
FROM {{tasks_table}} t
JOIN {{accounts_table}} a ON t.account_id = a.id
WHERE AI_FILTER(PROMPT('Does this sales activity mention competitors, competitive evaluation, vendor comparison, or selection process? {0}', t.description))
ORDER BY t.activity_date DESC
LIMIT 30;


# 🎯 MEDDIC Analysis with AI Intelligence

MEDDIC (Metrics, Economic Buyer, Decision Criteria, Decision Process, Implicate Pain, Champion) is a proven B2B sales methodology. Here we use AI to automatically extract MEDDIC elements from sales activities and assess deal health.

This transforms subjective deal assessment into data-driven insights.

In [ ]:
SELECT
  o.name AS opportunity_name,
  a.name AS account_name,
  o.stage_name,
  o.amount,
  o.close_date,
  AI_FILTER (
    PROMPT (
      'Does this opportunity show clear metrics or ROI discussions? Look for financial impact, cost savings, revenue growth targets. {0}',
      (SELECT LISTAGG (description, ' | ') FROM AI_SOL.SALES.TASKS AS t WHERE t.what_id = o.id)
    )
  ) AS has_metrics,
  AI_FILTER (
    PROMPT (
      'Is there evidence of economic buyer engagement? Look for C-level involvement, budget approval discussions, or executive sponsorship. {0}',
      (SELECT LISTAGG (description, ' | ') FROM AI_SOL.SALES.TASKS AS t WHERE t.what_id = o.id)
    )
  ) AS has_economic_buyer,
  AI_FILTER (
    PROMPT (
      'Are decision criteria clearly defined? Look for evaluation requirements, technical specifications, or vendor comparison criteria. {0}',
      (SELECT LISTAGG (description, ' | ') FROM AI_SOL.SALES.TASKS AS t WHERE t.what_id = o.id)
    )
  ) AS has_decision_criteria,
  AI_FILTER (
    PROMPT (
      'Is the decision process mapped out? Look for timeline discussions, approval workflows, or procurement processes. {0}',
      (SELECT LISTAGG (description, ' | ') FROM AI_SOL.SALES.TASKS AS t WHERE t.what_id = o.id)
    )
  ) AS has_decision_process,
  AI_FILTER (
    PROMPT (
      'Are there clear pain points or business challenges identified? Look for problems, inefficiencies, or urgent business needs. {0}',
      (SELECT LISTAGG (description, ' | ') FROM AI_SOL.SALES.TASKS AS t WHERE t.what_id = o.id)
    )
  ) AS has_implicated_pain,
  AI_FILTER (
    PROMPT (
      'Is there evidence of a champion or internal advocate? Look for stakeholder support, internal selling, or advocacy behaviors. {0}',
      (SELECT LISTAGG (description, ' | ') FROM AI_SOL.SALES.TASKS AS t WHERE t.what_id = o.id)
    )
  ) AS has_champion
FROM
  AI_SOL.SALES.OPPORTUNITIES AS o
  JOIN AI_SOL.SALES.ACCOUNTS AS a ON o.account_id = a.id
WHERE
  NOT o.stage_name IN ('Closed Won', 'Closed Lost')
ORDER BY
  o.amount DESC
LIMIT
  10;

# ✉️ AI-Generated Personalized Outreach

One of the most time-consuming tasks for sales reps is crafting personalized follow-up emails. Using the conversation history and context, we can generate relevant, personalized outreach that references specific discussions and next steps.

This demonstrates how AI can augment human creativity while maintaining authenticity.


In [ ]:
# Personalized Outreach Generator
st.header("✉️ AI-Powered Outreach Generator")

with st.form("outreach_form"):
    st.markdown("### Generate personalized follow-up emails based on recent interactions")
    
    # Get accounts with recent activity
    recent_accounts = session.sql(f"""
        SELECT DISTINCT 
            a.name,
            MAX(t.activity_date) as last_activity
        FROM {accounts_table} a
        JOIN {tasks_table} t ON a.id = t.account_id
        GROUP BY a.name
        ORDER BY last_activity DESC
        LIMIT 20
    """).to_pandas()
    
    selected_account_outreach = st.selectbox(
        "Select Account for Outreach:",
        options=recent_accounts['NAME'].tolist()
    )
    
    outreach_type = st.selectbox(
        "Outreach Type:",
        ["Follow-up after meeting", "Next steps confirmation", "Proposal follow-up", "Check-in email", "Meeting request"]
    )
    
    tone = st.selectbox(
        "Tone:",
        ["Professional", "Friendly", "Executive", "Technical"]
    )
    
    generate_outreach = st.form_submit_button("Generate Outreach Email")

if generate_outreach:
    # Get recent activity and context for the selected account
    outreach_context = session.sql(f"""
        WITH recent_activities AS (
            SELECT 
                t.description,
                t.subject,
                t.activity_date,
                t.type
            FROM {tasks_table} t
            JOIN {accounts_table} a ON t.account_id = a.id
            WHERE a.name = '{selected_account_outreach}'
            ORDER BY t.activity_date DESC
            LIMIT 5
        ),
        account_context AS (
            SELECT 
                a.name as account_name,
                a.industry,
                a.annual_revenue,
                c.first_name || ' ' || c.last_name as primary_contact,
                c.title,
                c.email
            FROM {accounts_table} a
            LEFT JOIN {contacts_table} c ON a.id = c.account_id
            WHERE a.name = '{selected_account_outreach}'
            AND c.is_primary_contact = TRUE
            LIMIT 1
        )
        SELECT 
            ac.account_name,
            ac.industry,
            ac.primary_contact,
            ac.title,
            ac.email,
            SNOWFLAKE.CORTEX.COMPLETE(
                'claude-4-sonnet',
                CONCAT(
                    'Write a {outreach_type} email in a {tone} tone to ', ac.primary_contact, ' (', ac.title, ') at ', ac.account_name, ' (', ac.industry, ' industry). ',
                    'Base the email on these recent interactions: ',
                    (SELECT LISTAGG(CONCAT(type, ': ', subject, ' - ', description), '; ') FROM recent_activities),
                    '. Make it personalized, relevant, and include appropriate next steps. Keep it concise (under 200 words). ',
                    'Include a clear subject line. Sign as the Snowball Analytics sales representative.'
                )
            ) as generated_email
        FROM account_context ac
    """).collect()
    
    if outreach_context:
        context = outreach_context[0]
        
        st.success(f"Outreach Generated for {context['ACCOUNT_NAME']}")
        
        # Display contact info
        col1, col2 = st.columns(2)
        with col1:
            st.markdown(f"**To:** {context['PRIMARY_CONTACT']}")
            st.markdown(f"**Title:** {context['TITLE']}")
        with col2:
            st.markdown(f"**Email:** {context['EMAIL']}")
            st.markdown(f"**Company:** {context['ACCOUNT_NAME']}")
        
        # Display generated email
        st.markdown("### 📧 Generated Email")
        
        email_content = context['GENERATED_EMAIL']
        
        # Make email editable
        edited_email = st.text_area(
            "Edit email before sending:",
            value=email_content,
            height=300
        )
        

# 🤖 Conversational Sales Assistant

The ultimate goal is natural language interaction with our sales data. Using Cortex Search, we can create a conversational assistant that answers complex sales questions, just like asking a senior sales analyst.

This represents the future of sales intelligence - instant, contextual, and conversational.


In [ ]:
-- Create Cortex Search service for sales data
-- This enables semantic search across all sales activities and relationships

CREATE OR REPLACE CORTEX SEARCH SERVICE SALES_SEARCH
  ON description
  ATTRIBUTES activity_date, account_id
  WAREHOUSE = tc_wh
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
  AS (
    SELECT
        t.id,
        t.activity_date,
        t.account_id,
        t.subject,
        t.type,
        t.description,
        a.name as account_name,
        a.industry
    FROM {{tasks_table}} t
    JOIN {{accounts_table}} a ON t.account_id = a.id
);


In [ ]:
# Conversational Sales Assistant
from snowflake.core import Root
from snowflake.cortex import complete
import json

st.header("🤖 Conversational Sales Assistant")
st.markdown("Ask natural language questions about accounts, opportunities, and sales activities.")

# Sample questions to inspire users
st.markdown("""
**Try asking questions like:**
- "What's the latest activity with Hamburg Digital?"
- "Show me deals where competitors were mentioned"
- "What accounts have budget approval discussions?"
- "Find me technical deep dives from last month"
- "Which accounts need follow-up this week?"
""")

root = Root(session)

# Conversational interface
with st.form("sales_query"):
    user_question = st.text_input(
        "Ask your sales question:",
        placeholder="What's the latest activity with Hamburg Digital?"
    )
    
    search_submitted = st.form_submit_button("Search")

if search_submitted and user_question:
    try:
        # Search sales data
        search_service = (root
            .databases["ai_sol"]
            .schemas["sales"]
            .cortex_search_services["sales_search"]
        )
        
        search_results = search_service.search(
            query=user_question,
            columns=["id", "description", "account_name", 
                    "type", "activity_date"],
            limit=5
        )
        
        # Generate AI response based on search results
        model = 'claude-4-sonnet'
        
        context_prompt = f"""
        Based on the following sales data, answer this question: "{user_question}"
        
        Sales Data:
        {search_results.to_str()}
        
        Provide a helpful, contextual answer that summarizes the relevant information and suggests next steps if appropriate. 
        Be specific about accounts, dates, and activities mentioned.
        """
        
        ai_response = complete(model, context_prompt)
        
        st.markdown("### 🎯 AI Response")
        st.markdown(ai_response)
        
        st.markdown("### 📋 Relevant Sales Activities")
        
        # Display search results in a more readable format
        results_json_string = search_results.to_json()  
        results_data = json.loads(results_json_string)  
        
        if results_data and 'results' in results_data:
            for i, result in enumerate(results_data['results'][:3]):  # Show top 3
                with st.expander(f"📌 {result.get('account_name', 'Unknown Account')} - {result.get('type', 'Activity')}"):
                    st.markdown(f"**Date:** {result.get('activity_date', 'N/A')}")
                    st.markdown(f"**Description:** {result.get('description', 'No description available')}")
        
    except Exception as e:
        st.error(f"Error: {str(e)}")
        

# 📊 AI for Business Intelligence

## 🎯 Purpose
Learn how to leverage Snowflake's AI capabilities for advanced business intelligence through semantic analysis.

## 🔑 Key Components

- **Analyst Tool**: Enables quantitative analysis through natural language queries
- **Data Semantics**: Ensures accuracy and consistency in analysis through defined relationships and metrics
- **Semantic Views**: Can be created through:
  - UI-based configuration in Snowflake interface
  - SQL-based definition (demonstrated in next cell)

## 💡 Benefits
- Natural language querying of your data
- Consistent metric definitions across your organization
- Enhanced data discoverability and understanding
- Improved data governance through semantic layer

In [ ]:
-- Sales Analytics Semantic View
-- This semantic view integrates CRM data for comprehensive sales performance analysis

CREATE OR REPLACE SEMANTIC VIEW sales_analytics 

-- Define the core tables for sales analysis
TABLES (
  accounts AS AI_SOL.SALES.ACCOUNTS 
  PRIMARY KEY (ID) 
  WITH SYNONYMS ('companies', 'customers', 'clients') 
  COMMENT = 'Company accounts and customer information',
  
  opportunities AS AI_SOL.SALES.OPPORTUNITIES 
  PRIMARY KEY (ID) 
  WITH SYNONYMS ('deals', 'sales opportunities', 'pipeline') 
  COMMENT = 'Sales opportunities and deal pipeline data',
  
  contacts AS AI_SOL.SALES.CONTACTS 
  PRIMARY KEY (ID) 
  WITH SYNONYMS ('people', 'leads', 'prospects') 
  COMMENT = 'Contact persons at customer accounts',
  
  tasks AS AI_SOL.SALES.TASKS 
  PRIMARY KEY (ID) 
  WITH SYNONYMS ('activities', 'sales activities', 'actions') 
  COMMENT = 'Sales activities and tasks'
)

-- Define relationships between tables
RELATIONSHIPS (
  opportunities_to_accounts AS 
    opportunities (ACCOUNT_ID) REFERENCES accounts,
  contacts_to_accounts AS 
    contacts (ACCOUNT_ID) REFERENCES accounts,
  tasks_to_accounts AS 
    tasks (ACCOUNT_ID) REFERENCES accounts,
  tasks_to_contacts AS 
    tasks (WHO_ID) REFERENCES contacts,
  tasks_to_opportunities AS 
    tasks (WHAT_ID) REFERENCES opportunities
)

-- Define facts for calculations
FACTS (
  opportunities.deal_amount AS AMOUNT,
  opportunities.probability_percentage AS PROBABILITY,
  accounts.company_revenue AS ANNUAL_REVENUE,
  accounts.employee_count AS NUMBER_OF_EMPLOYEES,
  tasks.call_duration_minutes AS CALL_DURATION_IN_SECONDS / 60.0 
    COMMENT = 'Call duration converted to minutes',
  tasks.activity_count AS 1 
    COMMENT = 'Count of individual activities'
) 

-- Define dimensions for grouping and filtering
DIMENSIONS (
  accounts.company_name AS NAME 
    WITH SYNONYMS ('account name', 'customer name') 
    COMMENT = 'Company or account name',
  accounts.industry_type AS INDUSTRY 
    WITH SYNONYMS ('sector', 'vertical') 
    COMMENT = 'Industry classification',
  accounts.company_rating AS RATING 
    WITH SYNONYMS ('account rating', 'customer tier') 
    COMMENT = 'Account rating (Hot, Warm, Cold)',
  accounts.country AS BILLING_COUNTRY 
    WITH SYNONYMS ('location', 'geography') 
    COMMENT = 'Account country location',
  accounts.currency_code AS CURRENCY_ISO_CODE 
    COMMENT = 'Currency for financial data',
    
  opportunities.deal_name AS NAME 
    WITH SYNONYMS ('opportunity name', 'deal name') 
    COMMENT = 'Name of the sales opportunity',
  opportunities.sales_stage AS STAGE_NAME 
    WITH SYNONYMS ('stage', 'pipeline stage') 
    COMMENT = 'Current stage in sales pipeline',
  opportunities.forecast_category AS FORECAST_CATEGORY_NAME 
    WITH SYNONYMS ('forecast type') 
    COMMENT = 'Forecast categorization',
  opportunities.lead_origin AS LEAD_SOURCE 
    WITH SYNONYMS ('source', 'lead channel') 
    COMMENT = 'Source of the lead or opportunity',
  opportunities.opportunity_type AS TYPE 
    COMMENT = 'Type of opportunity (New Customer, Existing Customer, etc.)',
  opportunities.close_date AS CLOSE_DATE 
    COMMENT = 'Expected or actual close date',
  opportunities.is_closed_won AS IS_WON 
    COMMENT = 'Whether opportunity was won',
  opportunities.is_closed_lost AS (IS_CLOSED = TRUE AND IS_WON = FALSE) 
    COMMENT = 'Whether opportunity was lost',
    
  contacts.contact_name AS NAME 
    WITH SYNONYMS ('person name', 'contact person') 
    COMMENT = 'Full name of the contact',
  contacts.job_title AS TITLE 
    WITH SYNONYMS ('position', 'role') 
    COMMENT = 'Job title or position',
  contacts.department_name AS DEPARTMENT 
    WITH SYNONYMS ('team', 'division') 
    COMMENT = 'Department or business unit',
  contacts.is_primary AS IS_PRIMARY_CONTACT 
    COMMENT = 'Whether this is the primary contact for the account',
    
  tasks.activity_subject AS SUBJECT 
    WITH SYNONYMS ('task subject', 'activity title') 
    COMMENT = 'Subject or title of the sales activity',
  tasks.activity_type AS TYPE 
    WITH SYNONYMS ('task type', 'activity category') 
    COMMENT = 'Type of sales activity',
  tasks.activity_status AS STATUS 
    COMMENT = 'Current status of the activity',
  tasks.priority_level AS PRIORITY 
    COMMENT = 'Priority level of the activity',
  tasks.activity_date AS ACTIVITY_DATE 
    COMMENT = 'Date when activity was performed',
  tasks.call_type AS CALL_TYPE 
    COMMENT = 'Type of call (Inbound, Outbound, Internal)',
  tasks.call_disposition AS CALL_DISPOSITION 
    WITH SYNONYMS ('call outcome', 'call result') 
    COMMENT = 'Outcome or disposition of the call'
) 

-- Define metrics for analysis
METRICS (
  -- Account metrics
  accounts.total_accounts AS COUNT(DISTINCT accounts.ID) 
    COMMENT = 'Total number of accounts',
  accounts.avg_annual_revenue AS AVG(accounts.company_revenue) 
    COMMENT = 'Average annual revenue of accounts',
  accounts.total_annual_revenue AS SUM(accounts.company_revenue) 
    COMMENT = 'Total annual revenue across all accounts',
    
  -- Opportunity metrics
  opportunities.total_opportunities AS COUNT(opportunities.ID) 
    COMMENT = 'Total number of opportunities',
  opportunities.total_pipeline_value AS SUM(opportunities.deal_amount) 
    COMMENT = 'Total value of all opportunities in pipeline',
  opportunities.avg_deal_size AS AVG(opportunities.deal_amount) 
    COMMENT = 'Average deal size across all opportunities',
  opportunities.weighted_pipeline AS SUM(opportunities.deal_amount * opportunities.probability_percentage / 100) 
    COMMENT = 'Pipeline value weighted by probability',
  opportunities.win_rate AS AVG(
    CASE 
      WHEN opportunities.IS_WON = TRUE THEN 1 
      WHEN opportunities.IS_CLOSED = TRUE THEN 0 
      ELSE NULL 
    END
  ) COMMENT = 'Win rate for closed opportunities',
  opportunities.total_won_value AS SUM(
    CASE 
      WHEN opportunities.IS_WON = TRUE THEN opportunities.deal_amount 
      ELSE 0 
    END
  ) COMMENT = 'Total value of won opportunities',
  opportunities.avg_sales_cycle AS AVG(
    CASE 
      WHEN opportunities.IS_CLOSED = TRUE 
      THEN DATEDIFF(day, opportunities.CREATED_DATE::DATE, opportunities.CLOSE_DATE::DATE)
      ELSE NULL 
    END
  ) COMMENT = 'Average sales cycle length in days for closed deals',
  
  -- Contact metrics
  contacts.total_contacts AS COUNT(contacts.ID) 
    COMMENT = 'Total number of contacts',
  contacts.contacts_per_account AS COUNT(contacts.ID) / COUNT(DISTINCT contacts.ACCOUNT_ID) 
    COMMENT = 'Average number of contacts per account',
    
  -- Activity metrics
  tasks.total_activities AS COUNT(tasks.ID) 
    COMMENT = 'Total number of sales activities',
  tasks.avg_call_duration AS AVG(tasks.call_duration_minutes) 
    COMMENT = 'Average call duration in minutes',
  tasks.total_call_time AS SUM(tasks.call_duration_minutes) 
    COMMENT = 'Total call time in minutes',
  tasks.activities_per_opportunity AS COUNT(tasks.ID) / COUNT(DISTINCT tasks.WHAT_ID) 
    COMMENT = 'Average number of activities per opportunity',
  tasks.completion_rate AS AVG(
    CASE 
      WHEN tasks.IS_CLOSED = TRUE THEN 1 
      ELSE 0 
    END
  ) COMMENT = 'Task completion rate'
) 

COMMENT = 'Comprehensive semantic view for sales analytics and CRM performance tracking'; 

In [ ]:
-- Semantic views can be used by AI features (Analyst & Intelligence)
-- or BI tools via SELECT *.. for example
-- this is how we'd return the opportunities by type and by how many activities have taken place

SELECT * FROM SEMANTIC_VIEW(
    sales_analytics  
    DIMENSIONS opportunities.deal_name, opportunities.opportunity_type
    METRICS tasks.total_activities
  );

# 🚀 Next Steps: From Demo to Production

Congratulations! You've experienced the power of AI-driven sales intelligence with Snowball Analytics. Here's how to take this from demo to production:

## 1. 🎯 **Deploy Conversational Assistant**
Integrate the search service and AI capabilities into your CRM or create a standalone Streamlit application for your sales team.

## 2. 📊 **Expand Semantic Views**
Head to **AI > Studio > Cortex Analyst** to create and update semantic views of your sales data, enabling natural language reporting and self-service analytics.

## 3. 🤖 **Build Sales Agents**
Use the search services and AI functions as tools for intelligent agents that can:
- Automatically prepare meeting briefs
- Monitor competitive threats
- Score deal health in real-time
- Generate personalized outreach at scale

## 4. 🔄 **Integrate with Your CRM**
Connect these AI capabilities directly to Salesforce, HubSpot, or your existing CRM through APIs for seamless workflow integration.

## 5. 📈 **Scale Across Sales Organization**
Roll out to your entire sales team with:
- Territory-specific insights
- Industry expertise augmentation  
- Automated coaching recommendations
- Real-time competitive intelligence

---

### 💡 **Key Takeaways**

✅ **Conversational AI** transforms how sales reps interact with data

✅ **Automated Intelligence** surfaces insights that would take hours to find manually

✅ **Contextual Recommendations** help reps take the right action at the right time

✅ **Competitive Awareness** keeps teams ahead of threats and opportunities

✅ **MEDDIC Automation** ensures consistent sales methodology across all deals

---

*Ready to revolutionize your sales intelligence? Start with your own data and see the transformation firsthand.*
